In [4]:
from utils.music_utils import *
from pathlib import Path
import sqlite3
import logging
from typing import Tuple, Optional

In [5]:
DATASET = Path("MegaSet")
SQLITE_INSTANCE = "music_09.db"
pkl_files = list(DATASET.rglob("*.pkl"))
valid_files = [check_file_info(pkl_file) for pkl_file in pkl_files]
invalid_files = [pkl_file for pkl_file, valid in zip(pkl_files, valid_files) if not valid]
print(f"Valid files: {len(valid_files)}")
print(f"Invalid files: {len(invalid_files)}")

Valid files: 16428
Invalid files: 0


In [7]:
# pick random invalid file and print_info
import random
if invalid_files:
    random_invalid_file = random.choice(invalid_files)
    print_info(random_invalid_file)
else:
    print("No invalid files found")

No invalid files found


In [8]:
def get_data(mp3_file: Path) -> dict:
    pkl_path = mp3_file.with_suffix(".pkl")
    with open(pkl_path, "rb") as f:
        data = pickle.load(f)
    data.pop("predictions_87", None)
    data.pop("embedding_512", None)
    data["mp3_path"] = str(mp3_file)
    return data

In [9]:
rdm_mp3 = pick_random_mp3(DATASET)
rdm_mp3_data = get_data(rdm_mp3)

print("\n".join(f"{key}: {value}" for key, value in rdm_mp3_data.items()))

filename: 08 Spectacle.mp3
filepath: MegaSet/Lyric Jones/Lyric Jones - 2014 - Love's Trail Mix/08 Spectacle.mp3
folder: MegaSet/Lyric Jones/Lyric Jones - 2014 - Love's Trail Mix
filesize: 3.99
title: Spectacle 
artist: Lyric Jones
album: Love's Trail Mix 
year: 2014
tracknumber: 8
genre: Hip Hop/Soul
top_5_genres: ['hiphop', 'rap', 'electronic', 'pop', 'triphop']
mp3_path: MegaSet/Lyric Jones/Lyric Jones - 2014 - Love's Trail Mix/08 Spectacle.mp3


In [10]:
def create_tables(self):
    try:
        with sqlite3.connect(self) as conn:
            c = conn.cursor()

            c.execute(
                """
                CREATE TABLE IF NOT EXISTS songs(
                    id INTEGER PRIMARY KEY,
                    filename TEXT NOT NULL,
                    filepath TEXT NOT NULL,
                    album_folder TEXT NOT NULL,
                    artist_folder TEXT NOT NULL,
                    filesize REAL NOT NULL,
                    title TEXT NOT NULL,
                    artist TEXT NOT NULL,
                    album TEXT NOT NULL,
                    year INTEGER,
                    tracknumber INTEGER,
                    genre TEXT,
                    top_5_genres TEXT
                )
                """
            )

    except sqlite3.Error as e:
        logging.error(f"An error occurred in create_tables: {e}")


create_tables(SQLITE_INSTANCE)

In [11]:
class Song:
    def __init__(
        self,
        filename: str,
        filepath: str,
        album_folder: str,
        artist_folder: str,
        filesize: float,
        title: str,
        artist: str,
        album: str,
        year: int,
        tracknumber: int,
        genre: str,
        top_5_genres: str,
    ):
        self.filename = filename
        self.filepath = filepath
        self.album_folder = album_folder
        self.artist_folder = artist_folder
        self.filesize = filesize
        self.title = title
        self.artist = artist
        self.album = album 
        self.year = year if year else None
        self.tracknumber = tracknumber  if tracknumber else None
        self.genre = genre  if genre else None
        self.top_5_genres = (
            top_5_genres if type(top_5_genres) == str else ", ".join(top_5_genres)
        )

class DatabaseManager:
    def __init__(self, db_path: str):
        self.conn = sqlite3.connect(db_path, check_same_thread=False)

    def close(self):
        self.conn.close()

    def execute(self, query: str, params: Tuple = ()) -> None:
        try:
            with self.conn:
                self.conn.execute(query, params)
                self.conn.commit()
        except sqlite3.Error as e:
            logging.error(f"An error occurred: {e}")

    def fetchone(self, query: str, params: Tuple = ()) -> Optional[Tuple]:
        try:
            with self.conn:
                return self.conn.execute(query, params).fetchone()
        except sqlite3.Error as e:
            logging.error(f"An error occurred: {e}")

    def fetchall(self, query: str, params: Tuple = ()) -> Optional[Tuple]:
        try:
            with self.conn:
                return self.conn.execute(query, params).fetchall()
        except sqlite3.Error as e:
            logging.error(f"An error occurred: {e}")

    def count_songs(self) -> int:
        query = "SELECT COUNT(*) FROM songs"
        return self.fetchone(query)[0]

    def insert_song(self, song: Song) -> None:
        song_data = (
            song.filename,
            song.filepath,
            song.album_folder,
            song.artist_folder,
            song.filesize,
            song.title,
            song.artist,
            song.album,
            song.year,
            song.tracknumber,
            song.genre,
            song.top_5_genres,
        )
        query = "SELECT * FROM songs WHERE filename = ? AND filepath = ?"
        existing_song = self.fetchone(query, (song.filename, song.filepath))
        if existing_song:
            return
        query = """
            INSERT INTO songs 
            (filename, filepath, album_folder, artist_folder, filesize, title, artist, album, year, tracknumber, genre, top_5_genres) 
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        """
        self.execute(query, song_data)

In [12]:
def pkl_to_dict(mp3_file: Path) -> Song:
    pkl_path = mp3_file.with_suffix(".pkl")
    with open(pkl_path, "rb") as f:
        data = pickle.load(f)
    data.pop("predictions_87", None)
    data.pop("embedding_512", None)
    data['artist_folder'] = data['filepath'].split("/")[1]
    song = Song(data["filename"], 
                data["filepath"], 
                data["folder"],
                data["artist_folder"],
                data["filesize"], 
                data["title"], 
                data["artist"], 
                data["album"],
                data["year"],
                data["tracknumber"],
                data["genre"],
                ",".join(data["top_5_genres"])
                )
    return song

In [13]:
db = DatabaseManager(SQLITE_INSTANCE)

# song = pkl_to_dict(rdm_mp3)
# db.insert_song(song)

In [14]:
db.fetchall("SELECT * FROM songs")[0]

IndexError: list index out of range

In [15]:
db.count_songs()

0

In [16]:
# loop over every file in DATASET and, read every .pkl into a Song object 
# and insert it into the database

for pkl_file in pkl_files:
    song = pkl_to_dict(pkl_file)
    db.insert_song(song)

# 27.7s

db.count_songs()

16428

In [17]:
# create a pandas dataframe from the database
import pandas as pd

query = "SELECT * FROM songs"
df = pd.read_sql_query(query, db.conn)

df.isna().sum()

id                  0
filename            0
filepath            0
album_folder        0
artist_folder       0
filesize            0
title               0
artist              0
album               0
year              137
tracknumber       129
genre            2126
top_5_genres        0
dtype: int64

In [19]:
## make sure the values for album don't contain any slashes

path_to_sqlite_db = 'music_09.db'
table_name = 'songs'

class Song:
    def __init__(self, filename: str, filepath: str, album_folder: str, artist_folder: str, filesize: float, title: str, artist: str, album: str, year: int, tracknumber: int, genre: str, top_5_genres: str):
        self.filename = filename
        self.filepath = filepath
        self.album_folder = album_folder
        self.artist_folder = artist_folder
        self.filesize = filesize
        self.title = title
        self.artist = artist
        self.album = album
        self.year = year
        self.tracknumber = tracknumber
        self.genre = genre
        self.top_5_genres = top_5_genres

        
def connect_to_db():
    conn = sqlite3.connect(path_to_sqlite_db)
    c = conn.cursor()
    return conn, c

def close_db(conn):
    conn.commit()
    conn.close()
    
def select_rows_with_slash_in_album():
    conn, c = connect_to_db()
    c.execute(f"SELECT * FROM {table_name} WHERE album LIKE '%/%'")
    rows = c.fetchall()
    close_db(conn)
    return rows


def count_rows_with_slash_in_album():
    conn, c = connect_to_db()
    c.execute(f"SELECT COUNT(*) FROM {table_name} WHERE album LIKE '%/%'")
    count = c.fetchone()[0]
    close_db(conn)
    return count

def update_album_column():
    conn, c = connect_to_db()

    c.execute(f"SELECT * FROM {table_name} WHERE album LIKE '%/%'")
    rows = c.fetchall()

    for row in rows:
        album = row[7]
        album = album.split('/')[-1]
        c.execute(f"UPDATE {table_name} SET album = ? WHERE filepath = ?", (album, row[2]))

    close_db(conn)


In [20]:
print(count_rows_with_slash_in_album())
update_album_column()
print(count_rows_with_slash_in_album())

229
0


In [ ]:
#######################################################################""

In [21]:
import musicbrainzngs

musicbrainzngs.set_rate_limit(limit_or_interval=1.0, new_requests=10)
# If you plan to submit data, authenticate
# musicbrainzngs.auth("user", "password")

# (this step is required, as per the webservice access rules
# at http://wiki.musicbrainz.org/XML_Web_Service/Rate_Limiting )
musicbrainzngs.set_useragent("Music_Recommandation_Milvus", "0.1", "https://github.com/Hatchi-Kin/music_similarity")

# If you are connecting to a different server
# musicbrainzngs.set_hostname("beta.musicbrainz.org")

In [22]:
try:
    result = musicbrainzngs.search_release_groups("Moby", "Play")
    print("Request was successful")
except musicbrainzngs.NetworkError:
    print("Service is unavailable")
except musicbrainzngs.MusicBrainzError as e:
    print(f"An error occurred: {e}")

result['release-group-list'][0]

Request was successful


{'id': '6838bf7e-b70d-3299-b9eb-8f5ec90d5229',
 'type': 'Album',
 'ext:score': '100',
 'title': 'Moby',
 'first-release-date': '1992-07-27',
 'primary-type': 'Album',
 'artist-credit': [{'name': 'Moby',
   'artist': {'id': '8970d868-0723-483b-a75b-51088913d3d4',
    'name': 'Moby',
    'sort-name': 'Moby',
    'disambiguation': 'electronic musician Richard Melville Hall',
    'alias-list': [{'locale': 'ja',
      'sort-name': 'モービー',
      'type': 'Artist name',
      'alias': 'モービー'},
     {'locale': 'en',
      'sort-name': 'DJ Moby',
      'type': 'Artist name',
      'alias': 'DJ Moby'},
     {'locale': 'en',
      'sort-name': 'Moby',
      'type': 'Artist name',
      'primary': 'primary',
      'alias': 'Moby'}]}}],
 'release-list': [{'id': '2587659e-7469-4a63-896c-3f4571b0689b',
   'title': 'Moby',
   'status': 'Official'},
  {'id': '45dff336-26bb-4e9a-8a9d-9c40b7f480bc',
   'title': 'Moby',
   'status': 'Official'},
  {'id': '51dd83d1-0803-480f-8d8f-9efddf7a42d2',
   'title': 

In [23]:
def get_album_id(artist, album):
    result = musicbrainzngs.search_releases(artist=artist, release=album)
    if result['release-list']:
        # If there's a result, take the first one
        first_result = result['release-list'][0]
        return first_result['id']
    else:
        return None

# Replace 'Artist Name' and 'Album Name' with the artist and album you're interested in
print(get_album_id('Moby', 'Play'))

07833ad1-9638-3127-804a-e5a7d4e2ad21


In [24]:
def get_album_year_by_id(album_id):
        release_info = musicbrainzngs.get_release_by_id(album_id, includes=["release-groups"])
        date = release_info['release']['release-group']['first-release-date']
        # The date is usually in the format 'YYYY-MM-DD', so we take the first 4 characters to get the year
        year = date[:4]
        return year

# Replace 'Artist Name' and 'Album Name' with the artist and album you're interested in
print(get_album_year_by_id('07833ad1-9638-3127-804a-e5a7d4e2ad21'))

1999


In [25]:
# for every row in sqlite that has a None value for year,
# get the album_id and then the year and update the row in sqlite and the pkl file

errors = 0
year_none = db.fetchall("SELECT * FROM songs WHERE year IS NULL")
for row in year_none:
    album_id = get_album_id(row[6], row[5])
    if album_id:
        year = get_album_year_by_id(album_id)
        db.execute("UPDATE songs SET year = ? WHERE filename = ? AND filepath = ?", (year, row[1], row[2]))
        pkl_path = Path(row[2]).with_suffix(".pkl")
        with open(pkl_path, "rb") as f:
            data = pickle.load(f)
        data["year"] = year
        with open(pkl_path, "wb") as f:
            pickle.dump(data, f)
    else:
        errors += 1

# 9m 5.8s

print(f"Number of errors: {errors}")

Number of errors: 55


In [26]:
# count the number of rows with a None value or an empty string for year

db.fetchall("SELECT COUNT(*) FROM songs WHERE year IS NULL OR year = '' OR year = ' '")

[(137,)]

In [27]:
# Get the list of rows of sqlite table where genre is None
# read the .pkl file and get the first element of the top_5_genres list
# and update the row's value for genre in sqlite

genre_none = db.fetchall("SELECT * FROM songs WHERE genre IS NULL OR genre = '' OR genre = ' ' or genre = 'None'")
for row in genre_none:
    song = pkl_to_dict(Path(row[2]))
    genre = song.top_5_genres.split(",")[0]
    db.execute("UPDATE songs SET genre = ? WHERE filename = ?", (genre, row[1]))

In [28]:
# df.isna().sum() equivalent

result = db.fetchall("PRAGMA table_info(songs)")
columns = [column[1] for column in result]

# For each column, count the number of rows with NULL values
for column in columns:
    count = db.fetchone(f"SELECT COUNT(*) FROM songs WHERE {column} IS NULL")[0]
    print(f"{column}: {count}")

id: 0
filename: 0
filepath: 0
album_folder: 0
artist_folder: 0
filesize: 0
title: 0
artist: 0
album: 0
year: 55
tracknumber: 129
genre: 0
top_5_genres: 0


In [29]:
# Get the list of rows is sqlite that have a NULL value for tracknumber

tracknumber_none = db.fetchall("SELECT * FROM songs WHERE tracknumber IS NULL OR tracknumber = '' OR tracknumber = ' '")

# print the first 5 rows of the list
for row in tracknumber_none[100:120]:
    # print(row[9])
    print(row[1])


A Conquest Of The Subconcious.mp3
Funky Junky.mp3
Gemalude.mp3
Intro.mp3
Untitled.mp3
Over You.mp3
Get Down.mp3
Classical (Outro).mp3
Flying-Turns-Cardboard-Lamb-Produced-By-Gesaffelstein (mp3-music.su).mp3
The-shoes-Cover-your-eyes-gesaffelstein-remix (mp3-music.su).mp3
ClashMusic-DjMix-Podcast-Gesaffelstein.mp3
Gesaffelstein-Liberte (mp3-music.su).mp3
Gesaffelstein-Start-Resistance (mp3-music.su).mp3
Gesaffelstein-Permanent-Vision (mp3-music.su).mp3
Gesaffelstein-Midnight-Anxiety-Theme (mp3-music.su).mp3
Limp Bizkit - I Would for You.mp3
Don't Worry Be Happy (Bobby McFerrin Cover).mp3
BLACKBIRD (The Beatles Cover).mp3
Grenade (Bruno Mars Cover).mp3
Little Boxes - Walk off the Earth.mp3


In [30]:
len(tracknumber_none)

129

In [32]:
# read the value for filename and see if there is a str with two successive numbers
# for example '05 - Les nuits parisiennes.mp3'. if there is, 
# extract the number and update the value for tracknumber in sqlite and the .pkl file
    
import re

for row in tracknumber_none:
    filename = row[1]
    tracknumber = re.findall(r"\b\d{2}\b", filename)
    if tracknumber:
        tracknumber = int(tracknumber[0])
        db.execute("UPDATE songs SET tracknumber = ? WHERE filename = ?", (tracknumber, row[1]))
        pkl_path = Path(row[2]).with_suffix(".pkl")
        with open(pkl_path, "rb") as f:
            data = pickle.load(f)
        data["tracknumber"] = tracknumber
        with open(pkl_path, "wb") as f:
            pickle.dump(data, f)


# read the value for filename and use re to check if the str contains 'Volo ' then a single digit*
# for example 'Volo 1-Jai beau.mp3'. if there is, extract the number and update the value for tracknumber in sqlite and the .pkl file
            
tracknumber_none = db.fetchall("SELECT * FROM songs WHERE tracknumber IS NULL OR tracknumber = '' OR tracknumber = ' '")
for row in tracknumber_none:
    filename = row[1]
    tracknumber = re.findall(r"Volo (\d)", filename)
    if tracknumber:
        tracknumber = int(tracknumber[0])
        db.execute("UPDATE songs SET tracknumber = ? WHERE filename = ?", (tracknumber, row[1]))
        pkl_path = Path(row[2]).with_suffix(".pkl")
        with open(pkl_path, "rb") as f:
            data = pickle.load(f)
        data["tracknumber"] = tracknumber
        with open(pkl_path, "wb") as f:
            pickle.dump(data, f)


In [33]:
# read the value for filename and use re to check if the str starts with a 
# single or double digit immediately followed by a dot then a space then a "
# for example '1. “Nightcall” - Kavinsky & Lovefoxxx.mp3' . If there is, 
# extract the number and update the value for tracknumber in sqlite and the .pkl file

tracknumber_none = db.fetchall("SELECT * FROM songs WHERE tracknumber IS NULL OR tracknumber = '' OR tracknumber = ' '")
for row in tracknumber_none:
    filename = row[1]
    tracknumber = re.findall(r"(\d{1,2})\. ", filename)
    if tracknumber:
        tracknumber = int(tracknumber[0])
        db.execute("UPDATE songs SET tracknumber = ? WHERE filename = ?", (tracknumber, row[1]))
        pkl_path = Path(row[2]).with_suffix(".pkl")
        with open(pkl_path, "rb") as f:
            data = pickle.load(f)
        data["tracknumber"] = tracknumber
        with open(pkl_path, "wb") as f:
            pickle.dump(data, f)


In [34]:
# df.isna().sum() equivalent

result = db.fetchall("PRAGMA table_info(songs)")
columns = [column[1] for column in result]

# For each column, count the number of rows with NULL values
for column in columns:
    count = db.fetchone(f"SELECT COUNT(*) FROM songs WHERE {column} IS NULL")[0]
    print(f"{column}: {count}")

id: 0
filename: 0
filepath: 0
album_folder: 0
artist_folder: 0
filesize: 0
title: 0
artist: 0
album: 0
year: 55
tracknumber: 129
genre: 0
top_5_genres: 0


In [35]:
# print the rows where value for tracknumber is NULL
tracknumber_none = db.fetchall("SELECT * FROM songs WHERE tracknumber IS NULL OR tracknumber = '' OR tracknumber = ' '")
for row in tracknumber_none:
    print(f"{row[1]}   -----   {row[6]}")

Sinkane - Hold Tight.mp3   -----   Sinkane - Hold Tight
Chinese man records - Bouncefloor.mp3   -----   Chinese man records - Bouncefloor
Chinese man records - BB's Electrify BUGS.mp3   -----   Chinese man records - BB's Electrify BUGS
Chinese man records - Searching for Hiero.mp3   -----   Chinese man records - Searching for Hiero
Chinese man records - Pudding à l'Arsenic.mp3   -----   Pudding à l'Arsenic
Peter Tosh - Rastafari Is.mp3   -----   Rastafari Is
Peter Tosh - Mama Africa.mp3   -----   Peter Tosh - Mama Africa
Magnetic Man -- Going Nowhere ft Alex Clare (Live Lounge!).mp3   -----   Magnetic Man -- Going Nowhere ft Alex Clare (Live Lounge!).mp3
Johnny Cash and June Carter - It Ain't Me, Babe.mp3   -----   Johnny Cash and June Carter - It Ain't Me, Babe
Johnny Cash & Bob Dylan - Complete Nashville Sessions - 1969.mp3   -----   AlbumWrap Album
Johnny Cash - Country Classics 1994 - Album.mp3   -----   All
Musica Country - Johnny Cash - Man In Black.mp3   -----   Man in Black
You

In [36]:
########### Unique artist

# for eah row in sqlite, read the filepath, extract the str between the third and fourth  / and compare to the value for artist_folder
# if different, add the path to a list
db = DatabaseManager(SQLITE_INSTANCE)

artist_diff = []
for row in db.fetchall("SELECT * FROM songs"):
    filepath = row[2]
    artist_folder = row[4]
    if filepath.split("/")[3] != artist_folder:
        artist_diff.append(filepath)

In [37]:
for art in artist_diff[40:70]:
    print(art)

MegaSet/Ska-P/Ska-P - 2008 - Lágrimas Y Gozos/02 - El Libertador.mp3
MegaSet/Ska-P/Ska-P - 2008 - Lágrimas Y Gozos/10 - El Tercero de la Foto.mp3
MegaSet/Ska-P/Ska-P - 2008 - Lágrimas Y Gozos/12 - Que Puedo Decir.mp3
MegaSet/Ska-P/Ska-P - 2008 - Lágrimas Y Gozos/03 - Crimen Sollicitationis.mp3
MegaSet/Ska-P/Ska-P - 2008 - Lágrimas Y Gozos/07 - El Imperio Caera.mp3
MegaSet/Ska-P/Ska-P - 2008 - Lágrimas Y Gozos/04 - Fuego y Miedo.mp3
MegaSet/Ska-P/Ska-P - 2008 - Lágrimas Y Gozos/08 - Los Hijos Bastardos de la Globalizacion.mp3
MegaSet/Ska-P/Ska-P - 2008 - Lágrimas Y Gozos/01 - Ni Fu Ni Fa.mp3
MegaSet/Ska-P/Ska-P - 2008 - Lágrimas Y Gozos/09 - Vandalo.mp3
MegaSet/Ska-P/Ska-P - 2008 - Lágrimas Y Gozos/06 - Gasta Claus.mp3
MegaSet/Ska-P/Ska-P - 2008 - Lágrimas Y Gozos/11 - Decadencia.mp3
MegaSet/Ska-P/Ska-P - 2000 - Planeta Eskoria/09 - Ska-P - A La Mierda.mp3
MegaSet/Ska-P/Ska-P - 2000 - Planeta Eskoria/10 - Ska-P - E.T.T.s.mp3
MegaSet/Ska-P/Ska-P - 2000 - Planeta Eskoria/07 - Ska-P - Eres

In [ ]:
######################################################################################################

In [38]:
db = DatabaseManager(SQLITE_INSTANCE)

query = "PRAGMA table_info(songs);"
columns_info = db.fetchall(query)
for column_info in columns_info:
    print(column_info)

(0, 'id', 'INTEGER', 0, None, 1)
(1, 'filename', 'TEXT', 1, None, 0)
(2, 'filepath', 'TEXT', 1, None, 0)
(3, 'album_folder', 'TEXT', 1, None, 0)
(4, 'artist_folder', 'TEXT', 1, None, 0)
(5, 'filesize', 'REAL', 1, None, 0)
(6, 'title', 'TEXT', 1, None, 0)
(7, 'artist', 'TEXT', 1, None, 0)
(8, 'album', 'TEXT', 1, None, 0)
(9, 'year', 'INTEGER', 0, None, 0)
(10, 'tracknumber', 'INTEGER', 0, None, 0)
(11, 'genre', 'TEXT', 0, None, 0)
(12, 'top_5_genres', 'TEXT', 0, None, 0)


In [39]:
query = "SELECT * FROM songs WHERE album LIKE '%/%'"
rows = db.fetchall(query)
for row in rows[:5]:
    print(f"Filename: {row[1]}\nFilepath: {row[2]}\nAlbum: {row[8]}\n")

In [40]:
for row in rows:
    # Split the album name on slashes and keep only the part after the last slash
    new_album = row[8].split('/')[-1]
    # Update the album column for this row
    query = f"UPDATE songs SET album = ? WHERE id = ?"
    db.execute(query, (new_album, row[0]))



In [41]:
query = "SELECT * FROM songs WHERE album LIKE '%/%'"
rows = db.fetchall(query)
for row in rows[:5]:
    print(f"Filename: {row[1]}\nFilepath: {row[2]}\nAlbum: {row[8]}\n")

In [42]:
# Fetch column information
query = "PRAGMA table_info(songs);"
columns_info = db.fetchall(query)

# Fetch the first row
query = "SELECT * FROM songs LIMIT 1"
row = db.fetchone(query)

# Pretty print column names and values
if row is not None:
    for i in range(len(row)):
        print(f"{columns_info[i][1]}: {row[i]}")
else:
    print("No rows found in the table")

id: 1
filename: 12 - Vous avez l'heure.mp3
filepath: MegaSet/Louise Attaque/Louise Attaque-1998-/12 - Vous avez l'heure.mp3
album_folder: MegaSet/Louise Attaque/Louise Attaque-1998-
artist_folder: Louise Attaque
filesize: 3.41
title: Vous avez l’heure
artist: Louise Attaque
album: Louise attaque
year: 1997
tracknumber: 12
genre: folk
top_5_genres: folk,country,pop,rock,popfolk


In [ ]:
# Fetch all rows
query = "SELECT * FROM songs"
rows = db.fetchall(query)

for row in rows:
    # Construct the full path for the artist folder
    new_artist_folder = 'MegaSet/' + row[4]
    # Update the artist_folder column for this row
    query = "UPDATE songs SET artist_folder = ? WHERE id = ?"
    db.execute(query, (new_artist_folder, row[0]))